In [70]:
#由于gii保存的都是值为0/1的ROI图片所以可以转换为png不受影响
import nibabel as nib 
import matplotlib.pyplot as plt 
import numpy as np 
import os
import pydicom

In [110]:
#读取roi 和 dicom 信息为矩阵
roi_path = "F:\GuiYiXray&MRI\X_ray&MRI\Her_2\Jiang Yuan Xiang/1/1.nii.gz"
roi_data = nib.load(roi_path).get_data()

ipykernel_launcher:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


In [112]:
#读取dicom图像的信息
dcm_path = "F:\GuiYiXray&MRI\X_ray&MRI\Her_2\Jiang Yuan Xiang/1/"
#读取dcm_path下每一个dicom格式的文件名
dirs = os.listdir(dcm_path)
dcm_names = []
i = 0
#读取.DCM的文件名
for file_name in dirs:
    if os.path.splitext(file_name)[1] =='.DCM':
        full_file_name = dcm_path + file_name 
        dcm_names.append(full_file_name)
#读取.DCM图像
dcms = []
for dcm_name in dcm_names:
    dcm = pydicom.dcmread(dcm_name).pixel_array
    dcms.append(dcm)


In [113]:
#将dicom原始图像和gii数组装换为numpy数组
rois = np.array(roi_data, dtype=np.int)
dcms = np.array(dcms, dtype=np.int)

In [114]:
print(rois.shape, dcms.shape)

(512, 512, 36) (36, 512, 512)


In [115]:
#提取minbox0(即最小外接矩形)
#先提取边框
def get_minbox0(gii):
    top_line = 0
    bottom_line = top_line + 1
    left_line = 0
    right_line = left_line + 1
    height, width = gii.shape 
    #寻找上下边界
    for i in range(height-1):
        if sum(gii[i,:]) == 0 and sum(gii[i+1,:]) > 0:
            top_line = i + 1
        if sum(gii[i,:]) > 0 and sum(gii[i+1,:]) == 0:
            bottom_line = i
    #寻找左右边界
    for j in range(width-1):
        if sum(gii[:,j]) == 0 and sum(gii[:,j+1]) > 0:
            left_line = j + 1
        if sum(gii[:,j]) > 0 and sum(gii[:,j+1]) == 0 :
            right_line = j
    return top_line, bottom_line, left_line, right_line 


                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [116]:
#将minbox0数组转化为png
from PIL import Image
n = rois.shape[2] #roi的数量
save_path = "F:\研究课题——乳腺癌分子分型分类\Data\MRI_ROI\min_box\Her2\Jiang Yuan Xiang/"
patient_name = 'Jiang Yuan Xiang'
for i in range(n):
    roi = np.transpose(rois[:,:,i])
    dcm = dcms[i]
    if np.max(roi) > 0:
        top_line, bottom_line, left_line, right_line = get_minbox0(roi)
        MINBOX0 = np.zeros([bottom_line - top_line + 1, right_line - left_line + 1])
        for j in range(top_line, bottom_line + 1):
            for k in range(left_line, right_line + 1):
                MINBOX0[j-top_line, k-left_line] = dcm[j,k]
        MINBOX0 = np.array(MINBOX0, dtype=np.int)
        im = Image.fromarray(MINBOX0)
        '''
        if im.mode == 'F':
            im = im.convert('L')
        '''
        im.save(save_path+patient_name+'-minbox0{}.png'.format(i))


